# Learning rate experiments for ADANNs
Adjusted from ADANN_semilinear_heat.ipynb

##### TODOs:

# Setup

We consider the semilinear heat PDE d dimensions:
$$
    \partial_t u (t, x)
=
    \nu (\Delta_{x} u)(t, x) + f(u(t, x)),
$$
for $(t, x) \in [0,T] \times [0, S]^d$ with periodic boundary conditions.

We want to approximate the map
$$
\Phi(u(0, \cdot)) = u(T, \cdot).
$$

Problem parameters:  $T, S, \nu \in (0,\infty)$, $f \colon \mathbb{R} \to \mathbb{R}$, and distribution of initial value.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import shutil
import importlib
import time 
import torch.nn
import seaborn as sns

sns.set_style("white")

sys.path.insert(1, '../1_Modules')

# Importing the modules
import random_function_generators
import ode_methods
import training
import training_samples_generators
import operator_learning_models
import utils
import semilinear_heat_multi_d_classical_methods
import evaluation_utils
import documentation_utils
import PDE_operations

sys.path.insert(1, '1_ADANN_Modules')

import ADANNs
import ADANNs_training
import ADANNs_grid
import ADANNs_opt
import ADANNs_lr_experiments

In [ ]:
# Reloading the modules
importlib.reload(random_function_generators)
importlib.reload(ode_methods)
importlib.reload(utils)
importlib.reload(training)
importlib.reload(training_samples_generators)
importlib.reload(operator_learning_models)
importlib.reload(semilinear_heat_multi_d_classical_methods)
importlib.reload(evaluation_utils)
importlib.reload(documentation_utils)
importlib.reload(PDE_operations)
importlib.reload(ADANNs)
importlib.reload(ADANNs_training)
importlib.reload(ADANNs_grid)
importlib.reload(ADANNs_opt)
importlib.reload(ADANNs_lr_experiments)


from random_function_generators import *
from ode_methods import *
from training import *
from training_samples_generators import *
from operator_learning_models import *
from utils import *
from semilinear_heat_multi_d_classical_methods import *
from evaluation_utils import *
from documentation_utils import *
from PDE_operations import *
from ADANNs import *
from ADANNs_training import *
from ADANNs_grid import *
from ADANNs_opt import *
from ADANNs_lr_experiments import *

### Setup

In [ ]:
test_run = False

# Problem setup for periodic semilinear PDE
###################################################
T = 2.
space_size = 1.
laplace_factor = 0.01
dim = 1
nonlin = lambda x : torch.sin(np.pi * x)
nonlin_name = "Sine"

# initial value
var = 10**5
decay_rate = 2
offset = np.power(var, 1/decay_rate)
inner_decay = 1.
initial_value_generator = RandnFourierSeriesGenerator([var, decay_rate, offset, inner_decay, space_size, dim])

In [ ]:
#Discretization operations
x_values = x_values_periodic
reduce_dimension = lambda values, space_resolution_step: reduce_dimension_periodic(values, space_resolution_step, dim=dim)
get_higher_nr_spacediscr = get_higher_nr_spacediscr_periodic
create_boundary_values = create_boundary_values_periodic

In [ ]:
# Name of the problem
pde_name = f"Semilinear_heat_{dim}-dimensional_T_{T}_space_size_{space_size}_laplace_factor_{laplace_factor}_nonlin_{nonlin_name}_var_{var}_decay_rate_{decay_rate}_offset_{offset}_inner_decay_{inner_decay}"

#Create folder for all outputs
output_folder_dir = create_output_folder(pde_name)
# output_folder_dir = f"Z Outputs/Learning rate experiment (server) - ZZ 2024-02-28 05h33m49s Semilinear_heat_1-dimensional_T_2.0_space_size_1.0_laplace_factor_0.01_nonlin_Sine_var_10000_decay_rate_2_offset_100.0_inner_decay_1.0/"

In [ ]:
generate_data = True 

#Method for reference solutions for training of models
reference_algorithm = lambda initial_values, nr_timesteps: periodic_semilinear_pde_spectral_lirk(initial_values, T, laplace_factor, nonlin, space_size, nr_timesteps, dim=dim)

# Train set parameters
train_space_resolution_step = 2 if test_run else (4 if dim==1 else 2)
train_nr_timesteps = 10 if test_run else 1000
nr_train_samples = 2**7 if test_run else (2**18 if dim==1 else 2**16)
nr_validation_samples = 2**7 if test_run else (2**14 if dim==1 else 2**11)

test_space_resolution_step = 2 if test_run else (8 if dim==1 else 4)
test_nr_timesteps = 20 if test_run else 1500
nr_test_samples = 2**7 if test_run else (2**14 if dim==1 else 2**11)


only_save_rough = True

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

parameters = {
    'T': T,
    'space_size': space_size,
    'laplace_factor': laplace_factor,
    'var': var,
    'dim': dim,
    'decay_rate': decay_rate,
    'offset': offset,
    'inner_decay': inner_decay,
    # 'nr_spacediscr': nr_spacediscr,
    'train_space_resolution_step': train_space_resolution_step,
    'train_nr_timesteps': train_nr_timesteps,
    'nr_train_samples': nr_train_samples,
    'nr_validation_samples': nr_validation_samples,
    'test_space_resolution_step': test_space_resolution_step,
    'nr_test_samples': nr_test_samples,
    'test_nr_timesteps': test_nr_timesteps,
    'reference_algorithm': reference_algorithm.__name__,
    'only_save_rough': only_save_rough
}

# save parametesr
with open(output_folder_dir + 'train_test_parameters.json', 'w') as fp:
    json.dump(parameters, fp)
    
# Optimizer
optimizer_class = torch.optim.Adam
# Loss function
loss_fn = torch.nn.MSELoss()

In [ ]:
# ADANNs Settings
ADANN_base_model_class = SecondOrderLirkFDMPeriodicSemilinearPDEAdannBasemodel
base_model_kwargs = {
    "T": T, 
    "laplace_factor": laplace_factor, 
    "nonlin": nonlin, 
    "space_size": space_size, 
    # "nr_spacediscr": nr_spacediscr, 
    "nonlin_name": nonlin_name, 
    "dim": dim
}

diff_model_class = ANNModel 
diff_model_params = [[1, 1, 1]]

#### Learning rate experiments

In [ ]:
# Setup
list_nr_timesteps = [2, 4, 8] if test_run else [2, 4, 8, 16, 32, 64]
list_nr_spacediscr = [4, 8, 16] if test_run else [4, 8, 16, 32, 64]

train_batchsize = 2**7 if test_run else (2**8 if dim==1 else 2**7)
nr_trainsteps =  1 if test_run else 50
nr_tests_per_lr = 2 if test_run else 5

smallest_power = -20
largest_power = 5.
maxiter = 1 if test_run else 15
base_lr_search_parameters = [nr_trainsteps, smallest_power, largest_power, maxiter]

In [ ]:
best_learning_rates = adanns_learning_rate(
    ADANN_base_model_class=ADANN_base_model_class,
    base_model_kwargs=base_model_kwargs,
    diff_model_class=diff_model_class,
    diff_model_params=diff_model_params,
    list_nr_timesteps=list_nr_timesteps,
    list_nr_spacediscr=list_nr_spacediscr,
    train_space_resolution_step=train_space_resolution_step,
    test_space_resolution_step=test_space_resolution_step,
    get_higher_nr_spacediscr=get_higher_nr_spacediscr,
    initial_value_generator=initial_value_generator,
    reference_algorithm=reference_algorithm,
    nr_train_samples=nr_train_samples,
    nr_validation_samples=nr_validation_samples,
    train_nr_timesteps=train_nr_timesteps,
    test_nr_timesteps=test_nr_timesteps,
    reduce_dimension=reduce_dimension,
    train_batchsize=train_batchsize,
    nr_tests_per_lr=nr_tests_per_lr,
    optimizer_class=optimizer_class,
    loss_fn=loss_fn,
    base_lr_search_parameters=base_lr_search_parameters,
    output_folder_dir=output_folder_dir,
)
print(best_learning_rates)

In [ ]:
plot_best_lr(output_folder_dir, shade="minmax")
# plot_best_lr(output_folder_dir,shade="stddev")